In [60]:
import json
import nltk

In [43]:
dataset = json.load(open('train-v1.1.json','r'))

In [59]:
qn, an = 0, 0
skipped = 0

In [49]:
def tokenize(sequence):
    tokens = [token.replace("``", '"').replace("''", '"') for token in nltk.word_tokenize(sequence)]
    return [x for x in tokens] # .encode('utf8')

In [51]:
def token_idx_map(context, context_tokens):
    acc = ''
    current_token_idx = 0
    token_map = dict()
    for char_idx, char in enumerate(context):
        if char != ' ':
            acc += char
            context_token = str(context_tokens[current_token_idx])
            if acc == context_token:
                syn_start = char_idx - len(acc) + 1
                token_map[syn_start] = [acc, current_token_idx]
                acc = ''
                current_token_idx += 1
    return token_map

In [55]:
train_data=[]

In [56]:
 for articles_id in range(len(dataset['data'])):
    article_paragraphs = dataset['data'][articles_id]['paragraphs']
    for pid in range(len(article_paragraphs)):
        context = article_paragraphs[pid]['context']
        # The following replacements are suggested in the paper
        # BidAF (Seo et al., 2016)
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')

        context_tokens = tokenize(context)
        answer_map = token_idx_map(context, context_tokens)

        qas = article_paragraphs[pid]['qas']
        for qid in range(len(qas)):
            question = qas[qid]['question']
            question_tokens = tokenize(question)

            answers = qas[qid]['answers']
            qn += 1

            num_answers = list(range(1))

            for ans_id in num_answers:
                # it contains answer_start, text
                text = qas[qid]['answers'][ans_id]['text']
                a_s = qas[qid]['answers'][ans_id]['answer_start']

                text_tokens = tokenize(text)

                answer_start = qas[qid]['answers'][ans_id]['answer_start']

                answer_end = answer_start + len(text)

                last_word_answer = len(text_tokens[-1]) # add one to get the first char

                try:
                    a_start_idx = answer_map[answer_start][1]

                    a_end_idx = answer_map[answer_end - last_word_answer][1]
                    
                    train_data.append([context_tokens,question_tokens,text_tokens,a_start_idx,a_end_idx])
                    
                    # remove length restraint since we deal with it later
#                     context_file.write(' '.join(context_tokens) + '\n')
#                     question_file.write(' '.join(question_tokens) + '\n')
#                     text_file.write(' '.join(text_tokens) + '\n')
#                     span_file.write(' '.join([str(a_start_idx), str(a_end_idx)]) + '\n')

                except Exception as e:
                    skipped += 1

                an += 1

print("Skipped {} question/answer pairs in {}".format(skipped, tier))

NameError: name 'tier' is not defined